In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from gensim.models import KeyedVectors




In [12]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = 'C:/Users/juver/Downloads/glove.6B/glove.6B.100d.txt'  # Update with your GloVe file path
tmp_file = 'C:/Users/juver/Downloads/glove.6B/glove.6B.100d.word2vec'

# Convert GloVe format to Word2Vec format
glove2word2vec(glove_file, tmp_file)

# Load GloVe word vectors in Word2Vec format
glove_model = KeyedVectors.load_word2vec_format(tmp_file, binary=False)
# Load GloVe word vectors
#glove_model = KeyedVectors.load_word2vec_format('C:/Users/juver/Downloads/glove.6B/glove.6B.100d.txt', binary=False)



C:\Users\juver\AppData\Local\Temp\ipykernel_8120\3314891050.py:8: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, tmp_file)


In [14]:
# Assuming 'Product Data' is the column containing your text data
df_total = pd.read_csv("ecommerceDataset.csv")
df_total["Label"] = df_total["Household"]
df_total["Product Data"] = df_total["Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and some for eternal bliss.so bring home this elegant print that is lushed with rich colors that makes it nothing but sheer elegance to be to your friends and family.it would be treasured forever by whoever your lucky recipient is. Liven up your place with these intriguing paintings that are high definition hd graphic digital prints for home, office or any room."]




In [16]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juver\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [17]:
# Split the data
_, df = train_test_split(df_total, test_size=0.02, random_state=42)

# Tokenize and preprocess
tokenized_paragraphs = [word_tokenize(doc.lower()) for doc in df['Product Data'].astype(str).tolist()]

# Function to generate GloVe embeddings for a document
def generate_glove_embeddings(document):
    embeddings = [glove_model[word] for word in document if word in glove_model]
    return embeddings



In [18]:
# Generate GloVe embeddings for each document
glove_embeddings = [generate_glove_embeddings(doc) for doc in tokenized_paragraphs]
df['GloVe Embeddings'] = glove_embeddings



In [30]:
# Continue with the rest of your code for model training and evaluation
import numpy as np
# Function to calculate the mean of embeddings
'''
def calculate_mean(lst):
    return np.mean(lst)
import numpy as np

def calculate_mean(lst):
    numeric_values = [value for value in lst if isinstance(value, (int, float))]
    return np.mean(numeric_values) if numeric_values else np.nan
'''
def calculate_mean(lst):
    if isinstance(lst, (int, float)):
        return lst
    numeric_values = [value for value in lst if isinstance(value, (int, float))]
    return np.mean(numeric_values) if numeric_values else np.nan


# Function to get train/test splits
def get_splits(embedding):
    X = df[embedding].apply(pd.Series)
    X = X.applymap(calculate_mean)
    X = X.fillna(0)
    y = df['Label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test



In [31]:
# Train and evaluate models
models = {'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
          'SVM': SVC(kernel='linear'),
          'Logistic Regression': LogisticRegression(max_iter=1000)}



In [32]:
from sklearn.metrics import accuracy_score, f1_score

def get_scores(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, f1


In [33]:
emb = df.columns[2:]

for embedding in emb:
    X_train, X_test, y_train, y_test = get_splits(embedding)
    for model_name, model in models.items():
        accuracy, f1 = get_scores(model, X_train, X_test, y_train, y_test)
        print(f'Accuracy with {embedding} trained on {model_name} model: {accuracy}')
        print(f'F1 Score with {embedding} trained on {model_name} model: {f1}')

Accuracy with Label trained on Random Forest model: 0.43564356435643564
F1 Score with Label trained on Random Forest model: 0.26439057698873336
Accuracy with Label trained on SVM model: 0.43564356435643564
F1 Score with Label trained on SVM model: 0.26439057698873336
Accuracy with Label trained on Logistic Regression model: 0.43564356435643564
F1 Score with Label trained on Logistic Regression model: 0.26439057698873336
Accuracy with Product Data trained on Random Forest model: 0.43564356435643564
F1 Score with Product Data trained on Random Forest model: 0.26439057698873336
Accuracy with Product Data trained on SVM model: 0.43564356435643564
F1 Score with Product Data trained on SVM model: 0.26439057698873336
Accuracy with Product Data trained on Logistic Regression model: 0.43564356435643564
F1 Score with Product Data trained on Logistic Regression model: 0.26439057698873336
Accuracy with GloVe Embeddings trained on Random Forest model: 0.43564356435643564
F1 Score with GloVe Embeddi